<a href="https://colab.research.google.com/drive/1VeolR4xuSSancsqd3KYTuefPh4IGB700?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy import stats
from scipy.stats import rankdata
import pickle
import mne
import os
import sys
import pandas as pd
from scipy.spatial.distance import cdist

sys.path.append(os.path.abspath("../src"))
import functions as fun

In [2]:
# URL to the stim_info.csv file on GitHub repository
path = '../data/'

# Load stimulation site information directly from the GitHub raw URL
stim_info = pd.read_csv(path+'stim_info.csv')
sub2use = 'sub-01'
run = 'run-01'

# Filter by subject and run, extract the stimulation coordinates, and compute the Euclidean distance between the stimulation site and each brain region centroid.
filtered_df = stim_info[(stim_info['Subject'] == sub2use) & (stim_info['Run'] == run)]
sito_stim = np.array([filtered_df['x_MNI'].values[0], filtered_df['y_MNI'].values[0], filtered_df['z_MNI'].values[0]])

In [23]:
percentages = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
ini_pre, fini_pre = 25, 250
ini_post, fini_post = 310, 900

stim_info = pd.read_csv(path + 'stim_info.csv')
filtered_df = stim_info[(stim_info['Subject'] == sub2use) & (stim_info['Run'] == run)]

baseline = (-0.3, -0.05)
eeg = mne.read_epochs(path + 'SEEG/sub1/' + sub2use + '_' + run + '_epoched.fif').filter(None, 40., h_trans_bandwidth='auto', filter_length='auto',phase='zero', verbose=False).apply_baseline(baseline)

Reading ../data/SEEG/sub1/sub-01_run-01_epoched.fif ...
    Found the data of interest:
        t =    -300.00 ...     700.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


<ipython-input-23-5f25a8a9fe09>:9: RuntimeWarning: This filename (../data/SEEG/sub1/sub-01_run-01_epoched.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  eeg = mne.read_epochs(path + 'SEEG/sub1/' + sub2use + '_' + run + '_epoched.fif').filter(None, 40., h_trans_bandwidth='auto', filter_length='auto',phase='zero', verbose=False).apply_baseline(baseline)


Applying baseline correction (mode: mean)


In [24]:
dat_wake=np.swapaxes(np.swapaxes(eeg.get_data(),0,2),0,1)
dat_wake=dat_wake[:,:-1,:]

idx2use = eeg.ch_names.index(np.array(filtered_df['Stimulation channels'])[0])

stim_coords = eeg.info['dig'][idx2use + 3]['r']*1000

In [25]:
percentage=percentages[-1]

distances = np.zeros((len(eeg.ch_names)))


for xx in range(len(eeg.ch_names)):
  ch2use = eeg.info['dig'][xx + 3]['r']*1000
  distances[xx] = np.sqrt(np.sum((stim_coords-ch2use)**2, axis=0))

ch_index2use = np.argsort(distances)[:int(distances.shape[0] * percentage / 100)]
R = int(distances[ch_index2use[-1]]) + 1

In [26]:
percentage

100

In [28]:
metrics_pre, metrics_post, Corr, Piva = fun.analyze_pre_VS_post(dat_wake, distances=distances, R=R, ini_pre=ini_pre, fini_pre=fini_pre, ini_post=ini_post, fini_post=fini_post)

data_save = {
   'metrics_pre': metrics_pre,
   'metrics_post': metrics_post,
   'Corr': Corr,
   'Piva': Piva
}

MetricsOfInterest loop through trials...
compute features for trial 0
compute features for trial 1
compute features for trial 2
compute features for trial 3
compute features for trial 4
compute features for trial 5
compute features for trial 6
compute features for trial 7
compute features for trial 8
compute features for trial 9
compute features for trial 10
compute features for trial 11
compute features for trial 12
compute features for trial 13
compute features for trial 14
compute features for trial 15
compute features for trial 16
compute features for trial 17
compute features for trial 18
compute features for trial 19
compute features for trial 20
compute features for trial 21
compute features for trial 22
compute features for trial 23
compute features for trial 24
compute features for trial 25
compute features for trial 26
compute features for trial 27
compute features for trial 28
compute features for trial 29
compute features for trial 30
compute features for trial 31
compute f

FileNotFoundError: [Errno 2] No such file or directory: '../data/SEEG/sub1/sub-01/sub-01_run-01_100_metrics.pkl'

In [32]:
with open(path + 'SEEG/sub1/' + sub2use  + '_' + run + '_Radius-' + str(percentage) + '_metrics.pkl', 'wb') as file:
   pickle.dump(data_save, file)